In [ ]:
import geopandas as gpd
import fiona
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

: 

In [ ]:
plt.rcParams["figure.figsize"] = (16, 12)

The data represented hereare the reuslt of a ~ 16-year effort to generate a vegetation map of Joshua Tree NP (https://irma.nps.gov/DataStore/Reference/Profile/2233319):

```
Inc. (AIS) out of Redlands, CA. The mapping effort began in 1996 and by 2004 they had produced a vegetation map (referred to as the 2005 version of the map), along with two reports (see Appendix F and G) titled, Photo-Interpretation Report, USGS-NPS Vegetation and Inventory and Mapping Program, Joshua Tree National Park and USGS-NPS Vegetation Mapping Program, Joshua Tree National Park Mapping Classification. AIS was hired again in 2009-2010 to assist in updating the map; they hosted the meeting in August 2009, then proceeded to make changes to the map as discussed at the meeting. For the most part, this involved revisiting aerial photos and reevaluating the map class assigned to each problematic polygon, as well as correcting any global recodes and minor edits to the nomenclature. Aerial imagery used for the project was from 1998, including the revisits in 2009, and the minimum mapping unit was defined as 0.50 hectares. For more detail on methods used by AIS to produce the map and a summary of the project pre-2005, refer to the reports mentioned above.
```

Due to the differing survey methodologies (ground surveys vs aerial photographs), there were discrepancies between land classifications outline here in the 'Accuracy Assessment Contigency Table' (https://irma.nps.gov/DataStore/Reference/Profile/2215775)

A report was created outlining the creation of this vegation map, as well (https://irma.nps.gov/DataStore/Reference/Profile/2215775)

![](jotrgeodata/JOTR%20GDB%20Diagram.jpg)

In [ ]:
fiona.listlayers("jotrgeodata/jotrgeodata.gdb")

For viz purposes, we will grab the park boundary:


In [ ]:
park_boundary = gpd.read_file("jotrgeodata/jotrgeodata.gdb", layer="JOTR_Park_Boundary")

For our initial purposes, we are interested primarily in the Veg polygon, we we will grab `JOTR_VegPolys`:

In [ ]:
veg_poly = gpd.read_file("jotrgeodata/jotrgeodata.gdb", layer="JOTR_VegPolys")

In [ ]:
veg_poly.head()

In [ ]:
veg_poly.MapUnit_Name.unique()

In [ ]:
[
    map_unit_name
    for map_unit_name in veg_poly.MapUnit_Name.unique()
    if "Joshua Tree" in map_unit_name
]

After doing some digging within the NPS documentation on stand metrics (https://irma.nps.gov/DataStore/DownloadFile/467293), we have identified a couple of `MapUnit_ID`s that appear to contain all three focal species, pinyon pine, california juniper, and joshua trees: 

In [ ]:
veg_poly.loc[veg_poly.Ma]

Let's engineer a feature for any MapUnit_Name that contains any Joshua Trees:

In [ ]:
veg_poly.loc[:, "contains_Joshua_Tree"] = [
    True if "Joshua Tree" in map_unit_name else False
    for map_unit_name in veg_poly.MapUnit_Name
]

In [ ]:
veg_poly.plot(column="contains_Joshua_Tree", legend=True)
park_boundary.plot(color="none", edgecolor="black", linewidth=2, ax=plt.gca())
plt.show()

In [ ]:
veg_poly.plot(column="contains_Joshua_Tree", legend=True)
park_boundary.plot(color="none", edgecolor="black", linewidth=2, ax=plt.gca())
plt.ylim(3.76e6, 3.77e6)
plt.xlim(580000, 590000)
plt.show()

For viz purposes, we'll engineer a coarser feature of association type - shrubland, woodland, herbaceous, dune/sand.

In [ ]:
def derive_assoc_type(map_unit_name):
    if "Woodland" in map_unit_name or "Forest" in map_unit_name:
        return "Woodland"
    elif "Shrubland" in map_unit_name:
        return "Shrubland"
    elif "Herbaceous" in map_unit_name:
        return "Herbaceous"
    elif map_unit_name in [
        "Rock Outcrops",
        "Dunes",
        "Non-vegetated Habitat (less than 2% absolute cover)",
        "Desert Twinbugs - Desert Sand Verbena Sparsely Vegetated Alliance (Desert Dunes and Sand Flats)",
        "Disturbed / Built-up",
    ]:
        return "Non-vegetated"
    elif map_unit_name in ["Playa", "Water"]:
        return "Water"
    else:
        return "Other"

In [ ]:
veg_poly.loc[:, "association_type"] = veg_poly.MapUnit_Name.apply(derive_assoc_type)

In [ ]:
cmap = ListedColormap(
    ["#F13FC4", "#A8A29C", "#B9DD97", "#28C4D5", "#A85B1F"], "indexed"
)

In [ ]:
veg_poly.plot(column="association_type", legend=True, cmap=cmap)
park_boundary.plot(color="none", edgecolor="black", linewidth=2, ax=plt.gca())
plt.show()

What is left over in 'other' - should be nothing?

In [ ]:
veg_poly[veg_poly.association_type == "Other"].MapUnit_Name.unique()

For Brookie / GEE mapping - a hex key associated with the list of map unit IDs that should be associated with that color:

#F13FC4 - `Herbaceous`

In [ ]:
veg_poly[veg_poly.association_type == "Herbaceous"].Poly_ID.to_list()

#A8A29C - `Non-vegetated`

In [ ]:
veg_poly[veg_poly.association_type == "Non-vegetated"].Poly_ID.to_list()

#B9DD97 - `Shrubland`

In [ ]:
veg_poly[veg_poly.association_type == "Shrubland"].Poly_ID.to_list()

#28C4D5 - `Water`

In [ ]:
veg_poly[veg_poly.association_type == "Water"].Poly_ID.to_list()

#A85B1F - `Woodland`

In [ ]:
veg_poly[veg_poly.association_type == "Woodland"].Poly_ID.to_list()

(Hello from gitpod env)